In [ ]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt




In [ ]:
'''
This script exctracts training variables from all logs from
tensorflow event files ("event*"), writes them to Pandas
and finally stores in long-format to a CSV-file including
all (readable) runs of the logging directory.
The magic "5" infers there are only the following v.tags:
[lr, loss, acc, val_loss, val_acc]
'''

import tensorflow as tf
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Get all event* runs from logging_dir subdirectories
logging_dir = './all_logs/'
event_paths = glob.glob(os.path.join(logging_dir,"event*"))


# Extraction function
def sum_log(path):
    #runlog = pd.DataFrame(columns=['metric', 'value'])
    logs = pd.DataFrame(columns=["step", "step_s", "D/loss_real","D/loss_fake","D/loss_cls","D/loss_gp","D/loss_total","G/loss_fake","G/loss_rec","G/loss_cls","G/loss_total"])
    content = {"step" : 0 , "step_s" : 0 , "D/loss_real":0.0,"D/loss_fake":0.0,"D/loss_cls":0.0,"D/loss_gp":0.0,"D/loss_total":0.0,"G/loss_fake":0.0,"G/loss_rec":0.0,"G/loss_cls":0.0,"G/loss_total":0.0}
    c = 0

    for e in tf.compat.v1.train.summary_iterator(path):
        c += 1
        content["step_s"] = int(e.step)
        content["step"] = int(e.step)
        for v in e.summary.value:
            content[v.tag] = np.fromstring(v.tensor.tensor_content,dtype=np.float32).item()
        if(c%9 == 0):

            logs = pd.concat([logs, pd.DataFrame([content.copy()])], ignore_index=True)


    # Dirty catch of DataLossError

    return logs

# Call & append
all_log = pd.DataFrame()
for path in event_paths:
    log = sum_log(path)
    if log is not None:
        if all_log.shape[0] == 0:
            all_log = log
        else:
            all_log = pd.concat([all_log,log], ignore_index = True)



all = all_log.copy()




In [ ]:

all_log = all.copy()
all_log = all_log.sort_values(by=['step_s'])
all_log = all_log.drop_duplicates("step")

for i in pd.DataFrame(all_log).columns:
    if(i == "step" or i == "step_s"):
        continue
    e = all_log.plot(x = 'step', y = i, subplots=True, figsize=(15,5), fontsize=17)[0].get_figure()
    e.savefig("Loss_visualization/" + i.split('/')[0] + "_" + i.split('/')[1] + ".jpg")



In [ ]:
log_2d = pd.DataFrame()
log_2d = sum_log("2d_log/events.out.tfevents.1713409479.e57bfb76452f.4583.0.v2")

all_log = all.copy()
log2 = log_2d.copy()

all_log = all_log.sort_values(by=['step_s'])
all_log = all_log.drop_duplicates("step")
all_log = all_log[all_log["step"] <= 10000]
log2 = log2.sort_values(by=['step_s'])
log2 = log2.drop_duplicates("step")

print(len(all_log))
print(len(log2))

for i in pd.DataFrame(all_log).columns:
    if(i == "step" or i == "step_s"):
        continue
    f = log2.plot.line(x = 'step', y = i, subplots=True, figsize=(15,5), fontsize=17, color="red")
    e = all_log.plot.line(x = 'step', y = i, subplots=True, figsize=(15,5), fontsize=17, ax = f)
    f = f[0].get_figure()
    f.savefig("compare_loss/" + i.split('/')[0] + "_" + i.split('/')[1] + "_compare.jpg")


